In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}|

<IPython.core.display.Javascript object>

In [2]:
#Import basic libraries
%matplotlib inline


import os
import sys
import settings
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from django_pandas.io import read_frame
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates

from matplotlib.dates import date2num

In [3]:
#Prepare to load stock data as pandas dataframe from source. In this case, prepare django
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from stocks.models import Listing, Stock

In [13]:
#Load data into dataframe
path = 'store_daily.h5'
store = pd.HDFStore(path)

df = store['df']
df.replace(np.nan,0, inplace=True)


In [15]:
df.tail()

,ABB,AEGISLOG,AMARAJABAT,AMBALALSA,HDFC,ANDHRAPET,ANSALAPI,UTIQUE,ARUNAHTEL,BOMDYEING,...,MADRASFERT,KOTAKBKETF,SETFNIFBK,SETFNIF50,ADVIKCA-RE,RAINBOWF-RE,ASMTEC-RE,HASJUICE,WEBSL,ADHIRAJ
date,,,,,,,,,,,,,,,,,,,,,
2022-06-09,2349.70,0.00,497.65,35.0,2265.55,0.00,0.00,0.00,0.00,113.60,...,49.15,355.01,350.71,169.24,0.0,0.0,0.0,0.0,0.0,0.0
2022-06-10,2338.90,228.25,493.80,0.0,2179.55,87.25,0.00,5.89,0.00,111.30,...,48.75,349.95,345.02,166.75,0.0,0.0,0.0,0.0,0.0,0.0
2022-06-13,2336.45,221.65,481.10,0.0,2131.20,84.45,13.70,5.57,11.75,102.80,...,45.30,338.75,335.55,162.70,0.0,0.0,0.0,0.0,0.0,0.0
2022-06-14,2320.35,222.80,482.00,32.6,2104.90,83.30,13.55,5.61,0.00,103.50,...,45.15,336.41,333.87,161.99,0.0,0.0,0.0,0.0,0.0,0.0
2022-06-15,2304.55,229.55,479.35,0.0,2105.45,83.50,13.83,0.00,11.93,103.65,...,44.90,336.59,333.75,161.78,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
correlation = df.corr(method ='pearson')
#correlation.replace(np.nan,0, inplace=True)
correlation.head()

,ABB,AEGISLOG,AMARAJABAT,AMBALALSA,HDFC,ANDHRAPET,ANSALAPI,UTIQUE,ARUNAHTEL,BOMDYEING,...,MADRASFERT,KOTAKBKETF,SETFNIFBK,SETFNIF50,ADVIKCA-RE,RAINBOWF-RE,ASMTEC-RE,HASJUICE,WEBSL,ADHIRAJ
ABB,1.000000,0.006309,-0.196190,0.213641,0.551772,0.708105,0.508880,0.642384,0.443110,0.323651,...,0.278267,0.714562,0.706930,0.704171,0.116474,0.164090,0.130854,NaN,NaN,NaN
AEGISLOG,0.006309,1.000000,0.324701,0.180265,0.187798,0.117286,-0.046167,-0.086644,0.127445,-0.001495,...,0.045157,-0.304117,-0.298429,-0.305854,-0.079660,-0.150133,-0.109699,NaN,NaN,NaN
AMARAJABAT,-0.196190,0.324701,1.000000,-0.080507,0.199189,-0.166078,-0.150346,-0.262908,-0.105456,-0.125116,...,-0.158885,-0.409759,-0.403925,-0.394662,-0.113486,-0.118606,-0.121514,NaN,NaN,NaN
AMBALALSA,0.213641,0.180265,-0.080507,1.000000,0.074648,0.257910,0.109123,0.243281,0.285698,0.127154,...,0.124395,0.039050,0.049997,0.047183,-0.033789,0.072596,0.109113,NaN,NaN,NaN
HDFC,0.551772,0.187798,0.199189,0.074648,1.000000,0.490356,0.323765,0.229810,0.309870,0.136027,...,0.014161,0.204268,0.208416,0.199667,0.016361,0.046046,0.047088,NaN,NaN,NaN


In [ ]:
import sys
!{sys.executable} -m pip install seaborn

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(18,18)})

In [ ]:
# plot the heatmap
# sns.heatmap(correlation, 
#         xticklabels=correlation.columns,
#         yticklabels=correlation.columns)
sns.heatmap(correlation, 
        xticklabels=False,
        yticklabels=False, mask=correlation.isnull())


In [ ]:
def plot_corr(df,size=10):
    '''Plot a graphical correlation matrix for a dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot'''
    
    %matplotlib inline
    import matplotlib.pyplot as plt

    # Compute the correlation matrix for the received dataframe
    corr = df.corr()
    
    corr.replace(np.nan,0, inplace=True)
    
    # Plot the correlation matrix
    fig, ax = plt.subplots(figsize=(size, size))
    cax = ax.matshow(corr, cmap='RdYlGn')
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90);
    plt.yticks(range(len(corr.columns)), corr.columns);
    
    # Add the colorbar legend
    cbar = fig.colorbar(cax, ticks=[-1, 0, 1], aspect=40, shrink=.8)

In [17]:
#https://github.com/TheLoneNut/CorrelationMatrixClustering/blob/master/CorrelationMatrixClustering.ipynb
import scipy
import scipy.cluster.hierarchy as sch


correlation.replace(np.nan,0, inplace=True)
#We use hierarchical clustering to determine which columns belongs to which cluster.
X = correlation.to_numpy() #correlation.values 
d = sch.distance.pdist(X)   # vector of ('columns' choose 2) pairwise distances
L = sch.linkage(d, method='complete')
ind = sch.fcluster(L, 0.25*d.max(), 'distance')


In [18]:
#Map the stock to its cluster
clusters = [ind[i] for i in list((np.argsort(ind)))]
columns = [df.columns.tolist()[i] for i in list((np.argsort(ind)))]

In [19]:
clustermap = zip(columns, clusters)
with open("clusters.csv", 'w') as fd:
    for r in clustermap:
        fd.write(f'{r[0]}, {r[1]}\n')

In [ ]:
df = df.reindex(columns, axis=1)

In [ ]:
#plot_corr(df, size=18)